In [13]:
import pandas as pd
import sqlite3

user_id: ID пользователя

started_at: Дата и время начала поездки

finished_at: Дата и время окончания поездки

cost: Стоимость поездки в копейках

paid_up: Статус оплаты поездки (True/False)

has_penalty: Информация о штрафе (NaN/значение)

platform: Платформа, с которой была совершена поездка

region_id: ID региона

In [9]:
#загрузка данных из csv файла
file_path = 'Whoosh_Trips_sql_test.csv'
whoosh_trips_data = pd.read_csv(file_path)
whoosh_trips_data.head()

,user_id,started_at,finished_at,cost,paid_up,has_penalty,platform,region_id
0,71506,2020-05-19 13:53:25.286+00,2020-05-19 14:13:25.62+00,19500,True,NaN,ANDROID,2
1,71506,2020-06-20 12:23:18.699+00,2020-06-20 12:50:35.368+00,24400,True,NaN,ANDROID,5
2,115258,2020-05-09 07:37:30.688+00,2020-05-09 08:36:48.3+00,43300,True,NaN,IOS,1
3,115258,2020-05-09 07:37:30.688+00,2020-05-09 08:36:48.3+00,43300,True,NaN,ANDROID,4
4,74999,2020-06-01 14:43:18.526+00,2020-06-01 15:08:46.742+00,18000,True,NaN,IOS,1


In [10]:
#cоздание базы данных в памяти
conn = sqlite3.connect(':memory:')

In [11]:
#cоздание таблицы SQL и загрузка данных
whoosh_trips_data.to_sql('whoosh_trips', conn, index=False, if_exists='replace')

63459

In [12]:
print(pd.read_sql('SELECT * FROM whoosh_trips limit 3', conn))

   user_id                  started_at                 finished_at   cost   
0    71506  2020-05-19 13:53:25.286+00   2020-05-19 14:13:25.62+00  19500  \
1    71506  2020-06-20 12:23:18.699+00  2020-06-20 12:50:35.368+00  24400   
2   115258  2020-05-09 07:37:30.688+00    2020-05-09 08:36:48.3+00  43300   

   paid_up has_penalty platform  region_id  
0        1        None  ANDROID          2  
1        1        None  ANDROID          5  
2        1        None      IOS          1  


Задача 1: Вывод размера среднего чека (в рублях) за все время.

Вычислим среднюю стоимость поездок, конвертировав стоимость из копеек в рубли

In [40]:
#средняя стоимость поездки в рублях
query = """
SELECT AVG(cost / 100.0) AS average_cost_rub
FROM whoosh_trips;
"""
result = pd.read_sql_query(query, conn)
print(result)

   average_cost_rub
0         202.16409


Задача 2: Вывод средней длительности поездки (в минутах) без штрафов (без penalty) для каждого региона (region_id) за все время.

Вычислим среднюю длительность поездок в минутах для каждого региона, исключая поездки с штрафами. Длительность поездки будет рассчитана как разница между finished_at и started_at

In [24]:
# Переводим столбцы с датами в формат datetime ISO8601
whoosh_trips_data['started_at'] = pd.to_datetime(whoosh_trips_data['started_at'], format='ISO8601', errors='coerce')
whoosh_trips_data['finished_at'] = pd.to_datetime(whoosh_trips_data['finished_at'], format='ISO8601', errors='coerce')
# Перезаписываем данные обратно в SQL таблицу
whoosh_trips_data.to_sql('whoosh_trips', conn, index=False, if_exists='replace')

In [26]:

# Запрос на получение средней продолжительности поездки в минутах
query_2 = """
SELECT region_id,
       AVG((strftime('%s', finished_at) - strftime('%s', started_at)) / 60.0) AS average_duration_minutes
FROM whoosh_trips
WHERE has_penalty IS NULL
GROUP BY region_id;
"""

average_duration_per_region = pd.read_sql(query_2, conn)
average_duration_per_region


,region_id,average_duration_minutes
0,1,26.892942
1,2,26.698551
2,3,27.165049
3,4,27.194798
4,5,27.238643


Задача 3: Вывод ID пользователя с наибольшим количеством оплаченных поездок (paid_up) без штрафов (без penalty) и количество его поездок за все время.

Для решения этой задачи, нужно сгруппировать данные по пользователям, посчитать количество оплаченных поездок без штрафов для каждого из них и выбрать пользователя с максимальным количеством таких поездок

In [38]:
#выводим id пользователя с наибольшим количеством оплаченных поездок
query_3 = """
SELECT user_id,
       COUNT(*) AS paid_trips_count
FROM whoosh_trips
WHERE paid_up = 1 AND has_penalty IS NULL
GROUP BY user_id
ORDER BY paid_trips_count DESC
LIMIT 1;
"""

top_user_paid_trips = pd.read_sql(query_3, conn)
top_user_paid_trips

,user_id,paid_trips_count
0,58024,190


Задача 4: Вывод общей суммы в рублях по каждой последней оплаченной поездке всех уникальных пользователей.

Для решения этой задачи, необходимо для каждого пользователя определить его последнюю оплаченную поездку и суммировать стоимости этих поездок

In [32]:
# вывод общей суммы по последним оплаченным поездкам в рублях
query_4 = """
WITH LastPaidTrips AS (
  SELECT user_id, MAX(started_at) AS last_trip_time
  FROM whoosh_trips
  WHERE paid_up = 1
  GROUP BY user_id
)
SELECT SUM(t.cost / 100.0) AS total_last_trips_cost_rub
FROM whoosh_trips AS t
JOIN LastPaidTrips AS lpt ON t.user_id = lpt.user_id AND t.started_at = lpt.last_trip_time;
"""

total_last_trips_cost = pd.read_sql(query_4, conn)
total_last_trips_cost_rub = total_last_trips_cost.at[0, 'total_last_trips_cost_rub']
total_last_trips_cost_rub

4345016.0

Задача 5: Вывод суммарной выручки в рублях от последних трёх поездок каждого уникального пользователя, совершившего не менее пяти поездок

Для решения этой задачи, нам нужно сначала выявить пользователей, которые совершили не менее пяти поездок, затем для каждого такого пользователя найти последние три поездки и суммировать их стоимость

In [39]:
# вывод суммарной выручки от последних трех поездок каждого пользователя.
query_5 = """
WITH RankedTrips AS (
  SELECT user_id,
         cost,
         ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY started_at DESC) AS trip_rank
  FROM whoosh_trips
),
UserTripCounts AS (
  SELECT user_id, COUNT(*) AS trip_count
  FROM whoosh_trips
  GROUP BY user_id
  HAVING trip_count >= 5
)
SELECT SUM(cost / 100.0) AS total_revenue_last_3_trips_rub
FROM RankedTrips
WHERE trip_rank <= 3 AND user_id IN (SELECT user_id FROM UserTripCounts);
"""

total_revenue_last_3_trips = pd.read_sql(query_5, conn)
total_revenue_last_3_trips_rub = total_revenue_last_3_trips.at[0, 'total_revenue_last_3_trips_rub']
total_revenue_last_3_trips_rub

1963683.0